In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#train_zip_new_data_merged_preprocessed.csv/
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_zip_preprocessed.csv")

In [ ]:
df.head()

,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,lst_pmnt_rcnc_d,balance,...,email,lk,assignee_npo,assignee_ops,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt,churn
0,2,1,2015,0.024783,-1,41,3492,3470,35,37379.76,...,0,-1,-1,-1,0.027582,1,1,0,1,0
1,1,2,2015,0.015819,-1,39,1191,244,947,176593.80,...,0,1,-1,-1,0.018477,1,-1,0,0,0
2,2,1,2022,0.013213,1,47,801,778,31,42401.23,...,-1,1,-1,-1,0.006976,1,1,0,1,0
3,1,2,2020,0.030976,1,48,7875,0,5844,2307.93,...,0,-1,-1,-1,0.027582,1,1,0,0,0
4,1,2,2012,0.013440,1,38,3452,0,2557,3757.16,...,-1,-1,-1,-1,0.006976,1,1,0,0,0


In [ ]:
df.drop(['Unnamed: 0', 'npo_account_id'], axis=1, inplace=True)

In [ ]:
ob_cols = df.select_dtypes('object').columns.to_list()
for col in ob_cols:
    if df[col].nunique() < 5:
        one_hot = pd.get_dummies(df[col], prefix=col, drop_first=True)
        df = pd.concat((df.drop(col, axis=1), one_hot), axis=1)
    else:
        mean_target = df.groupby(col)['churn'].mean()
        df[col] = df[col].map(mean_target)

In [ ]:
X = df.drop('churn', axis=1)
Y = df['churn']

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
lgb.fit(X_train, Y_train)

[LightGBM] [Info] Number of positive: 1297, number of negative: 78703
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013851 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3601
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016213 -> initscore=-4.105627
[LightGBM] [Info] Start training from score -4.105627


LGBMClassifier()

In [ ]:
y_pred = lgb.predict(X_test)

print(classification_report(Y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9933    0.9972    0.9953     19676
           1     0.7773    0.5926    0.6725       324

    accuracy                         0.9907     20000
   macro avg     0.8853    0.7949    0.8339     20000
weighted avg     0.9898    0.9907    0.9900     20000



In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, Y_train)
y_pred = xgb_model.predict(X_test)

print(classification_report(Y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9935    0.9973    0.9954     19676
           1     0.7831    0.6019    0.6806       324

    accuracy                         0.9909     20000
   macro avg     0.8883    0.7996    0.8380     20000
weighted avg     0.9901    0.9909    0.9903     20000



In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_zip_new_data_merged_preprocessed.csv")

In [ ]:
df1.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df1.head()

,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,lst_pmnt_rcnc_d,oprtn_sum_per_qrtr,oprtn_sum_per_year,...,fact_addrss,appl_mrkr,evry_qrtr_pmnt,churn,npo_sum_doxod,accnt_pnsn_schm,house_price,bezrabotica,potreb_price_x,inflycia
0,1,2,2015,0.015819,-1,39,1191,947,10646.13,104242.44,...,-1,0,0,0,0.00,62.0,99.7,5.6,100.69,15.416667
1,2,1,2022,0.013213,1,47,801,31,4878.05,19512.20,...,1,0,1,0,0.00,11.0,121.0,3.9,103.10,17.213333
2,1,2,2020,0.030976,1,48,7875,5844,88.76,88.76,...,1,0,0,0,88.76,354.0,112.0,5.8,100.36,4.023333
3,1,2,2012,0.013440,1,38,3452,2557,217.67,217.67,...,1,0,0,0,217.66,0.0,110.7,5.5,100.45,6.533333
4,2,1,2018,0.018502,-1,48,804,28,4181.24,25125.36,...,1,0,1,0,0.00,351.0,106.3,4.8,100.35,2.396667


In [ ]:
ob_cols = df1.select_dtypes('object').columns.to_list()
for col in ob_cols:
    if df1[col].nunique() < 5:
        one_hot = pd.get_dummies(df1[col], prefix=col, drop_first=True)
        df = pd.concat((df1.drop(col, axis=1), one_hot), axis=1)
    else:
        mean_target = df1.groupby(col)['churn'].mean()
        df1[col] = df1[col].map(mean_target)

In [ ]:
X1 = df1.drop('churn', axis=1)
Y1 = df1['churn']
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y1, random_state=0, test_size=0.2)

In [ ]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train1, Y_train1)
y_pred = xgb_model.predict(X_test1)

print(classification_report(Y_test1, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9931    0.9978    0.9955     19688
           1     0.8037    0.5641    0.6629       312

    accuracy                         0.9910     20000
   macro avg     0.8984    0.7810    0.8292     20000
weighted avg     0.9902    0.9910    0.9903     20000



In [ ]:
lgb = LGBMClassifier()
lgb.fit(X_train1, Y_train1)

y_pred = lgb.predict(X_test1)

print(classification_report(Y_test1, y_pred, digits=4))

[LightGBM] [Info] Number of positive: 1309, number of negative: 78691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015607 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3535
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016362 -> initscore=-4.096265
[LightGBM] [Info] Start training from score -4.096265
              precision    recall  f1-score   support

           0     0.9931    0.9974    0.9952     19688
           1     0.7709    0.5609    0.6494       312

    accuracy                         0.9906     20000
   macro avg     0.8820    0.7791    0.8223     20000
weighted avg     0.9896    0.9906    0.9898     20000



In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = xgb.XGBClassifier()

# Определение сетки гиперпараметров для перебора
param_grid = {
    'max_depth': [9, 11, 13],
    'n_estimators': [200, 250, 300],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Поиск лучших гиперпараметров
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, Y_train)

# Вывод наилучших гиперпараметров и оценки
print("Наилучшие параметры:", grid_search.best_params_)
print("Наилучшая оценка точности:", grid_search.best_score_)


Наилучшие параметры: {'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 200}
Наилучшая оценка точности: 0.9909000054659746


In [ ]:
xgb_model = xgb.XGBClassifier(max_depth=9, n_estimators=200, learning_rate=0.1)
xgb_model.fit(X_train, Y_train)
y_pred = xgb_model.predict(X_test)

print(classification_report(Y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9938    0.9975    0.9956     19676
           1     0.8016    0.6235    0.7014       324

    accuracy                         0.9914     20000
   macro avg     0.8977    0.8105    0.8485     20000
weighted avg     0.9907    0.9914    0.9909     20000



In [ ]:
model = LGBMClassifier()

# Определение сетки гиперпараметров для перебора
param_grid = {
    'max_depth': [ 13, 15, 17],
    'n_estimators': [200, 250, 300],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Поиск лучших гиперпараметров
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, Y_train)

# Вывод наилучших гиперпараметров и оценки
print("Наилучшие параметры:", grid_search.best_params_)
print("Наилучшая оценка точности:", grid_search.best_score_)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1297, number of negative: 78703
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012373 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3601
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016213 -> initscore=-4.105627
[LightGBM] [Info] Start training from score -4.105627
Наилучшие параметры: {'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 300}
Наилучшая оценка точности: 0.9906125010908301


In [ ]:
model = LGBMClassifier(max_depth=15, n_estimators=300, learning_rate=0.1)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

print(classification_report(Y_test, y_pred, digits=4))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1297, number of negative: 78703
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3601
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016213 -> initscore=-4.105627
[LightGBM] [Info] Start training from score -4.105627
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
              precision    recall  f1-score   support

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 3.3 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
# Определение модели CatBoostClassifier
model = CatBoostClassifier()

# Определение сетки гиперпараметров для перебора
param_grid = {
    'depth': [6],
    'iterations': [400, 450, 500, 550, 600],
    'learning_rate': [0.1]
}

# Поиск лучших гиперпараметров
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, Y_train)

# Вывод наилучших гиперпараметров и оценки
print("Наилучшие параметры:", grid_search.best_params_)
print("Наилучшая оценка точности:", grid_search.best_score_)


0:	learn: 0.4660926	total: 21ms	remaining: 8.4s
1:	learn: 0.3210351	total: 42.5ms	remaining: 8.46s
2:	learn: 0.2200820	total: 62.8ms	remaining: 8.3s
3:	learn: 0.1556719	total: 83.3ms	remaining: 8.25s
4:	learn: 0.1150775	total: 104ms	remaining: 8.19s
5:	learn: 0.0907001	total: 124ms	remaining: 8.15s
6:	learn: 0.0725673	total: 147ms	remaining: 8.22s
7:	learn: 0.0627773	total: 171ms	remaining: 8.36s
8:	learn: 0.0559770	total: 195ms	remaining: 8.45s
9:	learn: 0.0509958	total: 219ms	remaining: 8.56s
10:	learn: 0.0473799	total: 249ms	remaining: 8.8s
11:	learn: 0.0435361	total: 274ms	remaining: 8.85s
12:	learn: 0.0419908	total: 297ms	remaining: 8.83s
13:	learn: 0.0405874	total: 320ms	remaining: 8.82s
14:	learn: 0.0389639	total: 342ms	remaining: 8.79s
15:	learn: 0.0380684	total: 376ms	remaining: 9.03s
16:	learn: 0.0374003	total: 397ms	remaining: 8.94s
17:	learn: 0.0367330	total: 418ms	remaining: 8.88s
18:	learn: 0.0360014	total: 439ms	remaining: 8.81s
19:	learn: 0.0354163	total: 462ms	remainin

In [ ]:
cboost = CatBoostClassifier(depth=6, iterations=400, learning_rate=0.1, verbose=0)

cboost.fit(X_train,
           Y_train)
y_pred = cboost.predict(X_test)

print(classification_report(Y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9938    0.9979    0.9958     19676
           1     0.8306    0.6204    0.7102       324

    accuracy                         0.9918     20000
   macro avg     0.9122    0.8091    0.8530     20000
weighted avg     0.9911    0.9918    0.9912     20000

